In [37]:
import numpy as np
import pandas as pd
import scipy.stats as ss
import category_encoders as ce
from sklearn.impute import SimpleImputer
from feature_engine.imputation import MeanMedianImputer
from sklearn.experimental import enable_iterative_imputer

In [3]:
columns = [f'A{i}' for i in range(1,17)]
data = pd.read_csv('../data/credit_approvel/crx.data',names=columns)
data.head(5)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [4]:
data.isna().mean()

A1     0.0
A2     0.0
A3     0.0
A4     0.0
A5     0.0
A6     0.0
A7     0.0
A8     0.0
A9     0.0
A10    0.0
A11    0.0
A12    0.0
A13    0.0
A14    0.0
A15    0.0
A16    0.0
dtype: float64

In [21]:
masked = data == '?'
masked.sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [22]:
data.replace(to_replace='?',value=np.nan,inplace=True)
data.isna().mean()

A1     0.017391
A2     0.017391
A3     0.000000
A4     0.008696
A5     0.008696
A6     0.013043
A7     0.013043
A8     0.000000
A9     0.000000
A10    0.000000
A11    0.000000
A12    0.000000
A13    0.000000
A14    0.018841
A15    0.000000
A16    0.000000
dtype: float64

In [5]:
data.dtypes

A1      object
A2      object
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14     object
A15      int64
A16     object
dtype: object

In [24]:
numeric_features = data.select_dtypes(include=np.number)
categorical_feature = data.select_dtypes(include=np.object)

numeric_features.shape, categorical_feature.shape

((690, 4), (690, 12))

In [25]:
numeric_features.head(5)

,A3,A8,A11,A15
0,0.000,1.25,1,0
1,4.460,3.04,6,560
2,0.500,1.50,0,824
3,1.540,3.75,5,3
4,5.625,1.71,0,0


In [26]:
numeric_features.describe().T

,count,mean,std,min,25%,50%,75%,max
A3,690.0,4.758725,4.978163,0.0,1.000,2.75,7.2075,28.0
A8,690.0,2.223406,3.346513,0.0,0.165,1.00,2.6250,28.5
A11,690.0,2.400000,4.862940,0.0,0.000,0.00,3.0000,67.0
A15,690.0,1017.385507,5210.102598,0.0,0.000,5.00,395.5000,100000.0


In [27]:
categorical_feature.head(5)

,A1,A2,A4,A5,A6,A7,A9,A10,A12,A13,A14,A16
0,b,30.83,u,g,w,v,t,t,f,g,00202,+
1,a,58.67,u,g,q,h,t,t,f,g,00043,+
2,a,24.50,u,g,q,h,t,f,f,g,00280,+
3,b,27.83,u,g,w,v,t,t,t,g,00100,+
4,b,20.17,u,g,w,v,t,f,f,s,00120,+


In [28]:
categorical_feature.A2.unique, categorical_feature.A14.unique

(<bound method Series.unique of 0      30.83
 1      58.67
 2      24.50
 3      27.83
 4      20.17
        ...  
 685    21.08
 686    22.67
 687    25.25
 688    17.92
 689    35.00
 Name: A2, Length: 690, dtype: object>,
 <bound method Series.unique of 0      00202
 1      00043
 2      00280
 3      00100
 4      00120
        ...  
 685    00260
 686    00200
 687    00200
 688    00280
 689    00000
 Name: A14, Length: 690, dtype: object>)

In [44]:
numeric_features.loc[:,'A2'] = categorical_feature['A2'].astype(dtype='float64')
numeric_features.loc[:,'A14'] = categorical_feature['A14'].astype(dtype='float64')
categorical_feature.drop(labels=['A2','A14'],axis=1,inplace=True)

c:\users\ashen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
numeric_features

,A3,A8,A11,A15,A2,A14
0,0.000,1.25,1,0,30.83,202.0
1,4.460,3.04,6,560,58.67,43.0
2,0.500,1.50,0,824,24.50,280.0
3,1.540,3.75,5,3,27.83,100.0
4,5.625,1.71,0,0,20.17,120.0
...,...,...,...,...,...,...
685,10.085,1.25,0,0,21.08,260.0
686,0.750,2.00,2,394,22.67,200.0
687,13.500,2.00,1,1,25.25,200.0
688,0.205,0.04,0,750,17.92,280.0


In [46]:
imputer = MeanMedianImputer(imputation_method='mean')
imputer.fit(numeric_features)
imputer.imputer_dict_

{'A3': 4.758724637681159,
 'A8': 2.223405797101449,
 'A11': 2.4,
 'A15': 1017.3855072463768,
 'A2': 31.56817109144543,
 'A14': 184.01477104874445}

In [47]:
numeric_features = imputer.transform(numeric_features)
numeric_features

,A3,A8,A11,A15,A2,A14
0,0.000,1.25,1,0,30.83,202.0
1,4.460,3.04,6,560,58.67,43.0
2,0.500,1.50,0,824,24.50,280.0
3,1.540,3.75,5,3,27.83,100.0
4,5.625,1.71,0,0,20.17,120.0
...,...,...,...,...,...,...
685,10.085,1.25,0,0,21.08,260.0
686,0.750,2.00,2,394,22.67,200.0
687,13.500,2.00,1,1,25.25,200.0
688,0.205,0.04,0,750,17.92,280.0


In [48]:
numeric_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A3      690 non-null    float64
 1   A8      690 non-null    float64
 2   A11     690 non-null    int64  
 3   A15     690 non-null    int64  
 4   A2      690 non-null    float64
 5   A14     690 non-null    float64
dtypes: float64(4), int64(2)
memory usage: 32.5 KB


In [49]:
categorical_feature.isna().sum().sort_values(ascending=False)

A1     12
A6      9
A7      9
A4      6
A5      6
A9      0
A10     0
A12     0
A13     0
A16     0
dtype: int64

In [50]:
categorical_feature.fillna(value='missing',inplace=True)

c:\users\ashen\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:4465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,
